# <a id='0'>Objective:</a>
Predict clients' repayment abilities to ensure that clients capable of repayment are not rejected and that loans are given with a principal, maturity, and repayment calendar that will empower their clients to be successful.

# <a id='0'>O.S.E.M.N </a> 
- <a href='#1'>1. Obtain Data: Gather Data & Setup up environment</a>
- <a href='#2'>2. Scrub Data: Data Prep</a>
- <a href='#3'> 3. Explore data: EDA / Feature Engineering</a>
- <a href='#4'> 4. Model Data: Model Data for Prediction & Cross Validate</a>
- <a href='#4'> 5. iNterpret Data: Give Insights</a>

## <a id='1'>1. Obtain Data</a>

In [19]:
import pandas as pd 

For column description for each dataset please refer to HomeCredit_columns_description.csv located:
https://www.kaggle.com/c/home-credit-default-risk/data.

<img src="https://storage.googleapis.com/kaggle-media/competitions/home-credit/home_credit.png" width="800"></img>

In [51]:
# Read in dataset
# Main Datasets
application_train = pd.read_csv('./data/application_train.csv')
application_test = pd.read_csv('./data/application_test.csv')
# Supplement Dataset
bureau = pd.read_csv('./data/bureau.csv')
bureau_balance = pd.read_csv('./data/bureau_balance.csv')
POS_CASH_balance = pd.read_csv('./data/POS_CASH_balance.csv')
credit_card_balance = pd.read_csv('./data/credit_card_balance.csv')
previous_application = pd.read_csv('./data/previous_application.csv')
installments_payments = pd.read_csv('./data/installments_payments.csv')

In [53]:
print('\033[1m'+'DATASET','(ROWS, COLUMNS)'+'\033[0m')
print('application_train',application_train.shape)
print('application_test',application_test.shape)
print('bureau',bureau.shape)
print('bureau_balance',bureau_balance.shape)
print('POS_CASH_balance',POS_CASH_balance.shape)
print('credit_card_balance',credit_card_balance.shape)
print('previous_application',previous_application.shape)
print('installments_payments',installments_payments.shape)

DATASET (ROWS, COLUMNS)
application_train (307511, 122)
application_test (48744, 121)
bureau (1716428, 17)
bureau_balance (27299925, 3)
POS_CASH_balance (10001358, 8)
credit_card_balance (3840312, 23)
previous_application (1670214, 37)
installments_payments (13605401, 8)


## <a id='2'>2. Scrub Data</a>

In [71]:
t = application_train
t.sort_values("TARGET",ascending=False).head(10)



,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
142676,265437,1,Cash loans,M,N,N,0,76500.0,495000.0,16096.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
71932,183409,1,Cash loans,F,N,Y,0,225000.0,247275.0,22810.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,5.0
58253,167527,1,Cash loans,M,Y,Y,3,150750.0,315000.0,34056.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
142665,265426,1,Cash loans,M,Y,Y,2,225000.0,473760.0,50400.0,...,0,0,0,0,0.0,0.0,0.0,1.0,1.0,2.0
176474,304496,1,Cash loans,F,N,Y,0,202500.0,781920.0,32998.5,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,7.0
216123,350418,1,Cash loans,M,N,Y,0,126000.0,458725.5,24147.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
270794,413914,1,Cash loans,M,Y,Y,0,135000.0,781920.0,26856.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
288388,434091,1,Cash loans,F,N,Y,1,135000.0,148365.0,12024.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
253947,393848,1,Cash loans,F,N,Y,0,112500.0,274500.0,21816.0,...,0,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0


		f. Add all missing values needed 
		g. Mean median mode
		h. Change all categories to the correct type
		i. Engineer columns - create new columns if possible from the data you have
		j. Change categorical columns to dummy variables
